## Summary

Calculate features using [AlphaFold](https://github.com/deepmind/alphafold).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_alphafold.ipynb)"

export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

# === Cedar ===
# p100
# sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-1000 --time 24:00:00 --gres=gpu:p100:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1001-2000 --time 24:00:00 --gres=gpu:p100:1 ../scripts/run_notebook_gpu.sh

# v100l
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=2001-3000 --time 24:00:00 --gres=gpu:v100l:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=3001-3800 --time 24:00:00 --gres=gpu:v100l:1 ../scripts/run_notebook_gpu.sh

# CPU
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=3801-4182 --time 72:00:00 ../scripts/run_notebook_cpu.sh


# === Graham ===
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-3500 --time 24:00:00 --gres=gpu:t4:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=181-200 --time 24:00:00 --gres=gpu:v100:1 ../scripts/run_notebook_gpu.sh
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=201-306 ../scripts/run_notebook_cpu.sh
```

---

## Imports

In [1]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("31_run_alphafold").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

/tmp


In [4]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

(None, None, None, None)

In [5]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "cagi6-sherloc"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
    )
    TASK_ID = 3500
    TASK_COUNT = 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_sherloc/input-data-gby-protein.parquet',
 3500,
 4182)

In [6]:
device = xla_bridge.get_backend().platform

device

'gpu'

## Workspace

### Initialize model

In [7]:
AlphaFold.load_model(device=device)

### Load data

In [8]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

4182

In [9]:
assert TASK_COUNT == pfile.num_row_groups

In [10]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [11]:
display(input_df.head(2))
print(len(input_df))

,protein_id,mutation_id,mutation,effect,sequence,structure,alignment
0,Q14191,"[NM_000553.4:c.487G>C, NM_000553.4:c.487G>T, N...","[D163H, D163Y, D163V, S1013T, N166S, L169Q, C1...","[Uncertain significance, Uncertain significanc...",MSEKKLETTAQQRKCPEWMNVQNKRCAVEERKACVRKSVFEDDLPF...,HEADER ...,"[>101\n, MSEKKLETTAQQRKCPEWMNVQNKRCAVEERKACVRK..."


1


In [12]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

['mutation_id', 'mutation', 'effect']

In [13]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [14]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [15]:
def worker(mutation, data):
    try:
        results = AlphaFold.analyze_mutation(f"A_{mutation}", data)
    except AlphaFoldAnalyzeError as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"alphafold_core_{key}": value for key, value in results.items()}
        return results

In [16]:
results = []
for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    data = AlphaFold.build(tup.sequence, ligand_sequence=None, msa=tup.alignment)

    for mutation_idx, mutation in enumerate(tqdm(tup.mutation, leave=False)):
        if not validate_mutation(mutation):
            continue

        result = worker(mutation, data)
        if result is None:
            continue

        results.append(
            {
                "protein_id": tup.protein_id,
                "mutation": mutation,
            }
            | {field: getattr(tup, field)[mutation_idx] for field in iterable_fields}
            | result
        )

results_df = pd.DataFrame(results)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmptqppvqky.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmptqppvqky.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmptqppvqky.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpa_oo7_ce.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpa_oo7_ce.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpa_oo7_ce.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [17]:
display(results_df.head(2))
print(len(results_df))

,protein_id,mutation,mutation_id,effect,alphafold_core_scores_residue_plddt_wt,alphafold_core_scores_protein_plddt_wt,alphafold_core_scores_protein_max_predicted_aligned_error_wt,alphafold_core_scores_proten_ptm_wt,alphafold_core_features_residue_experimentally_resolved_wt,alphafold_core_features_residue_predicted_lddt_wt,...,alphafold_core_features_residue_experimentally_resolved_mut,alphafold_core_features_residue_predicted_lddt_mut,alphafold_core_features_residue_msa_first_row_mut,alphafold_core_features_residue_single_mut,alphafold_core_features_residue_structure_module_mut,alphafold_core_features_protein_experimentally_resolved_mut,alphafold_core_features_protein_predicted_lddt_mut,alphafold_core_features_protein_msa_first_row_mut,alphafold_core_features_protein_single_mut,alphafold_core_features_protein_structure_module_mut
0,Q14191,D163H,NM_000553.4:c.487G>C,Uncertain significance,90.357381,65.68561,31.75,0.523186,"[1.9947292, 2.098181, 2.4169307, 1.8486521, 2....","[-6.795537, -7.8329773, -6.9705763, -6.339246,...",...,"[1.9572178, 2.0707793, 2.3481874, 1.7782127, 2...","[-6.7607937, -7.8313513, -6.95637, -6.3245573,...","[0.53617436, 0.27606463, -15.891765, 1.4338753...","[23.178827, 37.773724, -6.383025, -15.528889, ...","[0.007426277, 0.002050683, -0.006094368, 0.002...","[1.3621839, 1.4258113, 1.6642208, 1.361827, 1....","[-5.772763, -6.300142, -5.5744143, -4.763823, ...","[1.0238394, 4.0296836, -0.7201518, 0.4787415, ...","[16.298262, 21.410181, 6.4630766, 0.75136036, ...","[0.005567466, 0.010155403, -0.0061244396, 0.00..."
1,Q14191,D163Y,NM_000553.4:c.487G>T,Uncertain significance,90.357381,65.68561,31.75,0.523186,"[1.9947292, 2.098181, 2.4169307, 1.8486521, 2....","[-6.795537, -7.8329773, -6.9705763, -6.339246,...",...,"[1.8958629, 2.0069966, 2.2594347, 1.7149972, 2...","[-6.755731, -7.8161335, -6.9484444, -6.3128376...","[0.2982066, -0.91982937, -16.13572, 1.734853, ...","[20.985367, 37.619385, -10.356445, -15.541066,...","[0.007068619, 0.0023403466, -0.006085431, 0.00...","[1.3617789, 1.4253657, 1.663696, 1.3614341, 1....","[-5.776592, -6.306531, -5.579072, -4.7677855, ...","[1.0214977, 4.028518, -0.7188361, 0.4797983, -...","[16.294582, 21.401125, 6.4536614, 0.75158364, ...","[0.005571265, 0.01018833, -0.0061233295, 0.003..."


3


### Save results

In [18]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold/cagi6-sherloc/shard-3500-of-4182.parquet')

In [19]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)